# Anomaly Detection (Trades) - Model Generation

We will use the [Support Vector Data Description (SVDD)](https://go.documentation.sas.com/?cdcId=pgmsascdc&cdcVersion=9.4_3.5&docsetId=casactml&docsetTarget=casactml_svdatadescription_details.htm&locale=en) algorithm, packaged in SAS Visual Data Mining and Machine Learning (VDMML), and deploy it using SAS Event Stream Processing ESPPy module.The CAS session is used to invoke Deep Learning to train a new model. to detect outliers in real time using streaming data. We will use also isolation forest available in VDMML.


### 0. Setup the Environment

First, import the necessary packages to run this notebook.

In [50]:
import os
import pandas as pd
import swat
import getpass

### 1. Start a SAS Viya CAS Session

The CAS session is used to invoke SAS Visual Data Mining and Machine Learning (VDMML) to train a new model.

In [51]:
os.environ["CAS_CLIENT_SSL_CA_LIST"] = "/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem"
password=getpass.getpass();

cashost='frasepviya35smp.cloud.com'
casport=5570
sess = swat.CAS(cashost, casport,'viyademo01',password)


sess.loadactionset(actionset="table")

 ······


NOTE: Added action set 'table'.


[actionset]

 'table'

+ Elapsed: 0.00032s, sys: 0.00029s, mem: 0.203mb

### 2. Data Preparation

Define the data to be used during model generation

In [52]:
# Name the existing CAS table to be used
indata = 'STREAMTOTALCOST_TRAIN'

# Locate CSV training data
csvsrvdir='/home/viyademo01/my_git/My_ESP_Demo/data'
csvfile='trade_events_training.csv'
results= sess.table.addCaslib(dataSource={"srcType":"PATH"},name="mycsvdir",path=csvsrvdir,subDirectories="True",activeonadd="false")

# Upload training data if not already existing in CAS
if not sess.table.tableExists(name=indata,caslib="public").exists:
    tbl = sess.table.loadTable(caslib="mycsvdir", path=csvfile, casout={"name":indata,"caslib":"public", "replace":True})

castbl = sess.CASTable(name=indata, caslib="public")

NOTE: Cloud Analytic Services added the caslib 'mycsvdir'.


In [10]:
sess.tableinfo(caslib="public")

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,STREAMTOTALCOST_TRAIN,75100,10,0,utf-8,2021-10-12T13:01:57+00:00,2021-10-12T13:01:57+00:00,2021-10-12T13:01:57+00:00,UTF8,1.949663e+09,0,0,0,trade_events_training.csv,mycsvdir,0,viyademo01,,2021-10-06T15:27:20+00:00,1.949153e+09
1,STREAMTRADES,29924,9,0,utf-8,2021-10-12T12:25:26+00:00,2021-10-12T12:33:49+00:00,2021-10-12T12:25:48+00:00,UTF8,1.949661e+09,0,0,0,,,0,viyademo01,,,NaN
2,STREAMTOTALCOST,29924,10,0,utf-8,2021-10-12T12:25:26+00:00,2021-10-12T12:33:49+00:00,2021-10-12T12:38:02+00:00,UTF8,1.949661e+09,0,0,0,,,0,viyademo01,,,NaN


In [43]:
castbl.describe()
list(castbl.columns)

['opCode',
 'opCode2',
 'tradeID',
 'security',
 'quantity',
 'price',
 'totalCost',
 'traderID',
 'time',
 'name']

### 4. Import SVDD action set

In [66]:
sess.loadactionset('svdd')

NOTE: Added action set 'svdd'.


[actionset]

 'svdd'

+ Elapsed: 0.00258s, user: 0.000422s, sys: 0.00209s, mem: 0.23mb

### 5. Train SVDD Model

SVDD algorithm is a one-class classification technique that is useful in applications where data that belongs to one class is abundant, but data about any other class is scarce or missing. Fraud detection, equipment health monitoring, and process control are some examples of application areas where the majority of the data belong to one class.

In its simplest form, an SVDD model is obtained by building a minimum-radius hypersphere around the one-class training data. The hypersphere provides a compact spherical description of the training data. This training data description can be used to determine whether a new observation is similar to the training data observations. The distance from any new observation to the hypersphere center is computed and compared with the hypersphere radius. If the distance is more than the radius, the observation is designated as an outlier. Using kernel functions in SVDD formulation provides a more flexible description of training data. Such description is nonspherical and conforms to the geometry of the data. PROC SVDD implements only the flexible data description.

We are using SVDD based K-charts to determine anomalous behavior in AHUs. K-chart is a nonparametric multivariate control chart that is used for statistical process control and can also be used for monitoring equipment health and operating data. It is implemented in two phases: In phase 1, observations from normal operations of the process are collected and are used to train a SVDD model and obtain the threshold r-square value.

In [67]:
# Phase 1: Model Training 
# Run svDataDescription.svddTrain action set on Training data 
sess.svDataDescription.svddTrain(tunemethod="MEAN2",
                         solver='actset',
                         inputs=[{"name":"totalcost"}],
                         id=['opCode','tradeID','security','quantity','price','totalCost','traderID','time','name'],
                         savestate = {"name": "trade_outliers_svdd", "replace":True},
                         table={"caslib":"public","name":"STREAMTOTALCOST_TRAIN"})

NOTE: Using Active Set Solver.
NOTE: SVDDTRAIN runs with default maxtime of 1800 seconds.
NOTE: Beginning data reading...
NOTE: Data reading complete.
NOTE: Starting processing for Model 1 (with bandwidth=72032.846453).
NOTE: Ending processing for Model 1 (with bandwidth=72032.846453).
NOTE: Output generation complete.
NOTE: Beginning save state generation...
NOTE: 6653 bytes were written to the table "trade_outliers_svdd" in the caslib "CASUSER(viyademo01)".
NOTE: Save state generation complete.


[NObs]

 Number of Observations
 
    RowId                         Type        N
 0  NREAD  Number of Observations Read  75100.0
 1  NUSED  Number of Observations Used  75100.0

[ModelInfo]

 Model Information
 
        RowId                   Description          Value        nValue
 0  OPTMETHOD           Optimization Method     Active Set           NaN
 1    KERTYPE                   Kernel Type            RBF           NaN
 2         BW          RBF Kernel Bandwidth   72032.846453  72032.846453
 3       TUNE    Bandwidth Selection Method  Modified Mean           NaN
 4   RELSCALE      Bandwidth Relative Scale   1.3851977652      1.385198
 5       FRAC     Expected Outlier Fraction           1E-6      0.000001
 6     OPTTOL        Optimization Tolerance         0.0001      0.000100
 7   NINTVARS  Number of Interval Variables              1      1.000000
 8   NNOMVARS   Number of Nominal Variables              0      0.000000

[TrainingResults]

 Training Results
 
     RowId                            Description      Value
 0     NSV              Number of Support Vectors  15.000000
 1    NSVB  Number of Support Vectors on Boundary  15.000000
 2  NDROBS         Number of Dropped Observations   0.000000
 3  THRESH               Threshold R Square Value   0.837365
 4     C_R                   Constant (C_r) Value   0.162635
 5   RTIME                     Run Time (seconds)   0.478408
 6    BCAL   Bandwidth Calculation Time (seconds)   0.000030

[OptSummary]

 Optimization Summary
 
        RowId           Description         Value    nValue
 0     NITERS  Number of Iterations             1  1.000000
 1        OBJ       Objective Value  0.1626350301  0.162635
 2      INFEA         Infeasibility  0.0000517012  0.000052
 3  OPTSTATUS   Optimization Status       Optimal       NaN
 4      DEGEN            Degenerate            No  0.000000

[StatusTable]

 Status Table
 
      RowId    Model   Status
 0  Model_1  Model 1  Success

+ Elapsed: 0.569s, user: 0.341s, sys: 0.432s, mem: 823mb

### 6. Score SVDD Model using ASTORE

This model of normal operations is then operationalized in phase 2 for anomaly detection. For each new observation, its distance value is computed and compared to the threshold r-square value. Observations for which distance value > threshold r-square generally indicate something abnormal in the process.

Load astore action set

In [46]:
sess.loadactionset('astore')

NOTE: Added action set 'astore'.


[actionset]

 'astore'

+ Elapsed: 0.00117s, user: 6.3e-05s, sys: 0.00108s, mem: 0.22mb

In [68]:
sess.aStore.describe(rstore={"name":'trade_outliers_svdd'}, epcode="TRUE")

[Key]

 Store Key
 
                                         Key
 0  5D96F2DC1330D6F6593B1E8A29B521718758E0C5

[Description]

 Basic Information
 
          Attribute               Value
 0  Analytic Engine                svdd
 1     Time Created  12Oct2021:17:29:01

[InputVariables]

 Input Variables
 
         Name  Length   Role      Type    RawType FormatName
 0  totalCost     8.0  Input  Interval        Num           
 1     opCode     1.0     Id            Character           
 2    tradeID     8.0     Id                  Num           
 3   security     3.0     Id            Character           
 4   quantity     8.0     Id                  Num           
 5      price     8.0     Id                  Num           
 6   traderID     8.0     Id                  Num           
 7       time     8.0     Id                  Num           
 8       name    11.0     Id            Character           

[OutputVariables]

 Output Variables
 
               Name  Length       Type          Label
 0           opCode     1.0  Character               
 1          tradeID     8.0        Num               
 2         security     3.0  Character               
 3         quantity     8.0        Num               
 4            price     8.0        Num               
 5        totalCost     8.0        Num               
 6         traderID     8.0        Num               
 7             time     8.0        Num               
 8             name    11.0  Character               
 9   _SVDDDISTANCE_     8.0        Num  SVDD Distance
 10     _SVDDSCORE_     8.0        Num     SVDD Score

[epcode]

 'data sasep.out;\n  dcl package score sc();\n  dcl double "totalCost";\n  dcl nchar(1) "opCode";\n  dcl double "tradeID";\n  dcl nchar(3) "security";\n  dcl double "quantity";\n  dcl double "price";\n  dcl double "traderID";\n  dcl double "time";\n  dcl nchar(11) "name";\n  dcl double "_SVDDDISTANCE_" having label n\'SVDD Distance\';\n  dcl double "_SVDDSCORE_" having label n\'SVDD Score\';\n  Keep \n    "opCode" \n    "tradeID" \n    "security" \n    "quantity" \n    "price" \n    "totalCost" \n    "traderID" \n    "time" \n    "name" \n    "_SVDDDISTANCE_" \n    "_SVDDSCORE_" \n    ;\n  varlist allvars[_all_];\n  method init();\n    sc.setvars(allvars);\n    sc.setKey(n\'5D96F2DC1330D6F6593B1E8A29B521718758E0C5\');\n  end;\n  method preScoreRecord();\n  end;\n  method postScoreRecord();\n  end;\n  method term();\n  end;\n  method run();\n    set sasep.in;\n    preScoreRecord();\n    sc.scoreRecord();\n    postScoreRecord();\n  end;\nenddata;\n'

+ Elapsed: 0.00376s, user: 0.00162s, sys: 0.00211s, mem: 8mb

In [47]:
sess.score(
     table={"name":'STREAMTOTALCOST_TRAIN', "caslib":'public'},
     casout='STREAMTOTALCOST_TRAIN_scored',
     rstore='trade_outliers_svdd'
    )

[OutputCasTables]

                 casLib                          Name   Rows  Columns                                           casTable
 0  CASUSER(viyademo01)  STREAMTOTALCOST_TRAIN_scored  75100       11  CASTable('STREAMTOTALCOST_TRAIN_scored', casli...

[Timing]

 Task Timing
 
                  Task   Seconds   Percent
 0   Loading the Store  0.000104  0.005432
 1  Creating the State  0.002264  0.118035
 2             Scoring  0.016489  0.859641
 3               Total  0.019181  1.000000

+ Elapsed: 0.0217s, user: 0.0712s, sys: 0.0142s, mem: 47.3mb

### 7. Generate an Analytic Store File

This ASTORE file can then be used in ESPPy for real-time anomaly detection.

In [181]:
#store=sess.download(rstore='svdd_ahu')
#with open('../ahu_svdd.astore','wb') as file:
#   file.write(store['blob'])

In [12]:
#sess.astore.describe(rstore='svdd_ahu')

In [34]:
sess.tableinfo(caslib='casuser')

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,_MODEL_7DAD3EF6-BE5C-1543-9F9B-3F9BCAC1924,81686,15,0,utf-8,2021-10-12T13:50:14+00:00,2021-10-12T13:50:14+00:00,2021-10-12T13:50:14+00:00,UTF8,1.949666e+09,0,0,0,,,0,viyademo01,,,NaN
1,TRADE_OUTLIERS_ISOLATION_FOREST,1,2,0,utf-8,2021-10-12T13:50:15+00:00,2021-10-12T13:50:15+00:00,2021-10-12T13:50:27+00:00,UTF8,1.949666e+09,0,0,0,,,0,viyademo01,,,NaN


In [14]:
castbls = sess.CASTable("STREAMTOTALCOST_TRAIN_scored")
castbls.head(10)

,_opcode,tradeID*,security,quantity,price,totalCost,traderID,time,name,_SVDDDISTANCE_,_SVDDSCORE_
0,ins,197416,SAP,3139,124.29500,390162.00500,10005,2020-07-20 10:40:25.213615,Jane Bloggs,0.838199,1.0
1,ins,197417,TPC,3841,113.69100,436687.13100,10003,2020-07-20 10:40:25.213652,Jane Doe,0.838143,-1.0
2,ins,197418,LPX,3291,131.87600,434003.91600,10005,2020-07-20 10:40:25.213845,Jane Bloggs,0.838139,-1.0
3,ins,197419,TPC,1826,13.68800,24994.28800,10003,2020-07-20 10:40:25.213940,Jane Doe,0.824121,-1.0
4,ins,197420,TPC,4116,127.33600,524114.97600,10005,2020-07-20 10:40:25.214141,Jane Bloggs,0.838260,1.0
5,ins,197421,IBM,3902,162.19000,632865.38000,10005,2020-07-20 10:40:25.214171,Jane Bloggs,0.838336,1.0
6,ins,197422,AOL,2437,89.45930,218012.31410,10002,2020-07-20 10:40:25.214355,John Doe,0.837747,-1.0
7,ins,197423,XOM,1887,62.18690,117346.68030,10002,2020-07-20 10:40:25.214379,John Doe,0.838016,-1.0
8,ins,197424,AOL,1337,187.95700,251298.50900,10003,2020-07-20 10:40:25.214657,Jane Doe,0.838261,1.0
9,ins,197425,AOL,4788,7.64216,36590.66208,10003,2020-07-20 10:40:25.214676,Jane Doe,0.822614,-1.0


### 8 Train an isolation forest model

In [76]:
sess.loadactionset('decisiontree')

sess.decisiontree.foresttrain(
    isolation='true',
    table={'name':'STREAMTOTALCOST_TRAIN', 'caslib':'Public'}, 
    inputs=[{'name':'totalCost'},{'name':'traderID'}],
    nBins=100,
    maxLevel=10, 
    maxBranch=2, 
    leafSize=1, 
    missing='USEINSEARCH', 
    vote='PROB', 
    minUseInSearch=1, 
    binOrder='true', 
    varImp='true', 
    casOut={'name':'_model_7dad3ef6-be5c-1543-9f9b-3f9bcac1924', 'caslib':'CASUSER', 'replace':'true'}, 
    mergeBin='true', 
    encodeName='true', 
    saveState={'name':'TRADE_OUTLIERS_ISOLATION_FOREST', 'caslib':'CASUSER', 'replace':'true'}, 
    nTree=100, 
    seed=12345,
    oob='true', 
    copyVars={"tradeID","security","quantity","price","totalCost","traderID","time","name"},
    target="totalCost"
)

NOTE: Added action set 'decisiontree'.
NOTE: 9294926 bytes were written to the table "TRADE_OUTLIERS_ISOLATION_FOREST" in the caslib "CASUSER(viyademo01)".


,Descr,Value
0,Number of Trees,100.00
1,Number of Selected Variables (M),1.00
2,Random Number Seed,12345.00
3,Training Sample Size for One Tree,100.00
4,Number of Bins,100.00
5,Number of Variables,2.00
6,Max Number of Tree Nodes,459.00
7,Min Number of Tree Nodes,9.00
8,Max Number of Branches,2.00
9,Min Number of Branches,2.00


In [74]:
sess.aStore.describe(rstore={"name":'TRADE_OUTLIERS_ISOLATION_FOREST'}, epcode="TRUE")

[Key]

 Store Key
 
                                         Key
 0  F74089C3E76056B219B41255292CC76C1B661DCD

[Description]

 Basic Information
 
          Attribute               Value
 0  Analytic Engine              forest
 1     Time Created  12Oct2021:17:36:11

[InputVariables]

 Input Variables
 
         Name  Length   Role      Type    RawType FormatName
 0  totalCost     8.0  Input  Interval        Num           
 1   traderID     8.0  Input  Interval        Num           
 2   quantity     8.0     Id                  Num           
 3      price     8.0     Id                  Num           
 4       time     8.0     Id                  Num           
 5   security    16.0     Id            Character           
 6    tradeID     8.0     Id                  Num           
 7       name    16.0     Id            Character           

[OutputVariables]

 Output Variables
 
         Name  Length       Type Label
 0   quantity     8.0        Num      
 1      price     8.0        Num      
 2   traderID     8.0        Num      
 3       time     8.0        Num      
 4   security    16.0  Character      
 5    tradeID     8.0        Num      
 6  totalCost     8.0        Num      
 7       name    16.0  Character      
 8  _Anomaly_     8.0        Num      

[epcode]

 'data sasep.out;\n  dcl package score sc();\n  dcl double "totalCost";\n  dcl double "traderID";\n  dcl double "quantity";\n  dcl double "price";\n  dcl double "time";\n  dcl nchar(16) "security";\n  dcl double "tradeID";\n  dcl nchar(16) "name";\n  dcl double "_Anomaly_";\n  Keep \n    "quantity" \n    "price" \n    "traderID" \n    "time" \n    "security" \n    "tradeID" \n    "totalCost" \n    "name" \n    "_Anomaly_" \n    ;\n  varlist allvars[_all_];\n  method init();\n    sc.setvars(allvars);\n    sc.setKey(n\'F74089C3E76056B219B41255292CC76C1B661DCD\');\n  end;\n  method preScoreRecord();\n  end;\n  method postScoreRecord();\n  end;\n  method term();\n  end;\n  method run();\n    set sasep.in;\n    preScoreRecord();\n    sc.scoreRecord();\n    postScoreRecord();\n  end;\nenddata;\n'

+ Elapsed: 0.1s, user: 0.1s, mem: 8.01mb

In [71]:
sess.score(
     table={"name":'STREAMTOTALCOST_TRAIN', "caslib":'public'},
     casout='STREAMTOTALCOST_TRAIN_scored',
     rstore='TRADE_OUTLIERS_ISOLATION_FOREST'
)

ERROR: The table STREAMTOTALCOST_TRAIN_scored already exists in the session.
ERROR: Action score failed.
ERROR: The action stopped due to errors.


+ Elapsed: 0.183s, user: 0.178s, sys: 0.0046s, mem: 16.1mb

### 9. Register model in model respository

Terminate the CAS session.

In [77]:
from sasctl import Session
from sasctl.tasks import register_model
from sasctl.services import model_repository,projects

sess.loadactionset('astore')

#modeltab='trade_outliers_svdd'
modeltab='trade_outliers_isolation_forest'
modellib='casuser'

astore = sess.CASTable(modeltab,caslib=modellib)

with Session('http://frasepviya35smp', 'viyademo01', password):
   try:
      existingProj=model_repository.get_project('Trade_ML002')
      model_repository.delete_project(existingProj)
   finally:
     model_repository.create_project('Trade_ML002','Public')
     register_model(astore, modeltab, 'Trade_ML002',)

NOTE: Added action set 'astore'.
NOTE: Added action set 'astore'.


IndexError: single positional indexer is out-of-bounds

### 9. Cleanup Your Project

Terminate the CAS session.

In [78]:
sess.close()